# 마스크 착용 여부 이미지 분류

이미지를 시각화하는게 필요해서 노트북으로 옮긴다. (220224 10:49)

## Import

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import albumentations
import albumentations.pytorch.transforms as A
import collections
import os
import random
import time
import copy

from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Subset
from torch.optim import lr_scheduler
from sklearn.model_selection import StratifiedShuffleSplit
from PIL import Image

## Seed 설정

In [3]:
def seed_set(SEED):
    random.seed(SEED)
    np.random.seed(SEED)
    os.environ["PYTHONHASHSEED"] = str(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

## Dataset

In [4]:
class CustomDataset(Dataset):
    def __init__(self, csv_path, kind=None, transform=None, train=True):
        self.kind = kind # mask, gender, age
        self.csv_path = csv_path
        self.transform = transform
        self.train = train
        self.df = pd.read_csv(self.csv_path)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        if self.train:
            img = Image.open(row['path'])
            label = row[self.kind]
            if self.transform:
                img = self.transform(image=np.array(img))['image']
        else:
            img = Image.open(row['ImageID'])
            label = row['ans']
        return img, label

    def __len__(self):
        return len(self.df)

In [120]:
def get_trainsforms():
    transforms = {
        'train': albumentations.Compose([A.ToTensorV2()]),
        'val': albumentations.Compose([A.ToTensorV2()])
    }
    return transforms

## Split train, validation

In [133]:
def split_train_valid(dataset, test_size=0.2, seed=0):
    y_train = [y for _, y in train_mask_data_set]
    counter_train = collections.Counter(y_train)
    print('original y label count', counter_mask_train)
    
    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=seed)
    indices = list(range(len(y_train_mask)))
    train_index, val_index = next(iter(sss.split(indices, y_train_mask)))
    
    train_ds = Subset(dataset, train_index)
    val_ds = Subset(dataset, val_index)
    y_train = [y for _, y in train_ds]
    y_val = [y for _, y in val_ds]
    counter_train = collections.Counter(y_train)
    counter_val = collections.Counter(y_val)
    print(counter_train)
    print(counter_val)
    return train_ds, val_ds

## MODEL

In [122]:
class CustomNewNet(nn.Module):
    def __init__(self, n_class):
        super(CustomNewNet, self).__init__()
        self.resnet34 = models.resnet34(pretrained=True)
        num_ftrs = self.resnet34.fc.in_features
        self.resnet34.fc = nn.Linear(num_ftrs, n_class)
    
    def forward(self, x):
        return self.resnet34(x)

## Train

In [154]:
def train_or_eval(model, data_loaders, criterion, optimizer, scheduler, n_epoch=10, device='cpu'):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(1, n_epoch + 1):
        print('Epoch {}/{}'.format(epoch, n_epoch))
        print('-' * 30)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            running_corrects = 0
            total = 0
            
            for imgs, labels in data_loaders[phase]:
                imgs = imgs.float().to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(imgs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase =="train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item()
                running_corrects += torch.sum(preds == labels.data)
                total += labels.size(0)

                if(phase == 'train'):
                    scheduler.step()

            epoch_loss = running_loss / len(data_loaders)
            epoch_acc = running_corrects.double() / total
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print()
            
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    model.load_state_dict(best_model_wts)
    return model


In [157]:
seed_set(SEED)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
EPOCH = 5
BATCH_SIZE = 16
LEARNING_RATE = 1e-3
SEED = 3086

model_ft = CustomNewNet(3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_ft.parameters(), lr=LEARNING_RATE)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [134]:
train_csv_path = '../input/data/train/full_path_three_label.csv'
train_mask_ds = MaskedFaceDataset(csv_path=train_csv_path, kind='mask', 
                                        transform=train_transform, train=True)
train_ds, val_ds = split_train_valid(train_mask_ds, 0.2, SEED)
data_loaders = {
    'train': DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4),
    'val': DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
}

original y label count Counter({0: 13500, 1: 2700, 2: 2700})
Counter({0: 10800, 1: 2160, 2: 2160})
Counter({0: 2700, 1: 540, 2: 540})


In [155]:
model_result = train_or_eval(model_ft, data_loaders, criterion, optimizer, scheduler, EPOCH, device)

Epoch 1/5
------------------------------
train Loss: 84.6439 Acc: 0.9483
val Loss: 15.6386 Acc: 0.9706

Epoch 2/5
------------------------------
train Loss: 85.0992 Acc: 0.9495
val Loss: 16.5106 Acc: 0.9656

Epoch 3/5
------------------------------
train Loss: 84.5187 Acc: 0.9481
val Loss: 15.5566 Acc: 0.9685

Epoch 4/5
------------------------------
train Loss: 86.3918 Acc: 0.9478
val Loss: 16.0216 Acc: 0.9690

Epoch 5/5
------------------------------
train Loss: 83.9424 Acc: 0.9477
val Loss: 16.1025 Acc: 0.9690

Training complete in 10m 35s


In [158]:
save_path = '../model'
torch.save({'epoch': EPOCH,
            'model_state_dict': model_result.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
           }, os.path.join(save_path, "resnet34_mask_model.pt"))

In [160]:
checkpoint = torch.load(os.path.join(save_path, "resnet34_mask_model.pt"))

In [3]:

import torch
torch.cuda.empty_cache()